# Processamento de Dados de Vendas (2022-2024)
Este notebook é dedicado ao tratamento abrangente de dados de vendas abrangendo o período de 2022 a 2024. Sua função principal é padronizar e corrigir os dados brutos de acordo com as regras comerciais estabelecidas, garantindo a integridade e a confiabilidade do conjunto de dados.

In [221]:
# Defininindo o arquivos de trabalho
arquivo = '../data/raw/vendas_22-24.csv'
arquivo_processado = '../data/processed/vendas_22-24.csv'

In [222]:
# Importação das bibliotecas utilizadas.
import pandas as pd
import statistics as sts

In [223]:
# Carregamento do arquivo de dados a ser utilizado.
dados = pd.read_csv(arquivo, delimiter=';')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             9672 non-null   object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       9691 non-null   float64
 5   Preço Unitário   9688 non-null   float64
 6   Total Venda      7979 non-null   float64
 7   Cliente_ID       9732 non-null   float64
 8   Forma_Pagamento  9684 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


In [224]:
# Verificando o padrão de colunas
dados.head()

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
0,2022-01-18,Loja B,Refrigerante,Bebidas,3.0,7.50,22.50,6727.0,Dinheiro
1,2024-12-29,Loja B,Leite,Alimentos,5.0,14.74,73.70,2120.0,Transferência
2,2023-03-21,Loja C,Refrigerante,Bebidas,2.0,NaN,72.60,5353.0,Transferência
3,2022-08-03,Loja C,Feijão,Alimentos,4.0,26.75,107.00,6835.0,Transferência
4,2022-06-29,Loja C,Feijão,Alimentos,7.0,30.47,213.29,5824.0,Dinheiro


In [225]:
# contagem de valores nulos.
dados.isnull().sum()

Data                  0
Loja                328
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID          268
Forma_Pagamento     316
dtype: int64

### Regra de negócio Loja

In [226]:
# Verificação dos dados de loja
dados.groupby('Loja').size()

Loja
Loja A    3238
Loja B    3215
Loja C    3219
dtype: int64

In [227]:
# Preenche o valor da loja com o valor de maior recorrência
dados.fillna({'Loja': 'Loja A'}, inplace=True)

### Regra de negócio Cliente

In [228]:
# Contagem de clientes nulos
dados['Cliente_ID'].isnull().sum()

np.int64(268)

In [229]:
# Preenchendo valores nulos com padrão
dados['Cliente_ID'] = dados['Cliente_ID'].fillna(0000)

In [230]:
# Contagem de clientes nulos
dados['Cliente_ID'].isnull().sum()

np.int64(0)

### Regra de negócio Quantidade

In [231]:
# Nova contagem de nulos
dados.isnull().sum()

Data                  0
Loja                  0
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID            0
Forma_Pagamento     316
dtype: int64

In [232]:
# Calculo da mediana para preenchimento de valores incorretos ou nulos
medianaQtd = sts.median(dados['Quantidade'])
medianaQtd

2.0

In [233]:
# Preenchimento de valores nulos
dados['Quantidade'] = dados['Quantidade'].fillna(medianaQtd)

In [234]:
# Filtro de valores fora da regra de negócio
dados['Quantidade'].loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

11     -4.0
67     -5.0
77     -4.0
166    -1.0
179    -5.0
       ... 
9867   -4.0
9873   -1.0
9888   -2.0
9985   -5.0
9987   -1.0
Name: Quantidade, Length: 367, dtype: float64

In [235]:
# Correção de valores incorretos com base na regra de negócio
dados.loc[((dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)), 'Quantidade'] = medianaQtd

In [236]:
# Nova verificação de valores fora da regra
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento


In [237]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             10000 non-null  object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       10000 non-null  float64
 5   Preço Unitário   9688 non-null   float64
 6   Total Venda      7979 non-null   float64
 7   Cliente_ID       10000 non-null  float64
 8   Forma_Pagamento  9684 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


In [238]:
dados = dados.astype({
    'Data': 'datetime64[us]',
    'Loja': 'string',
    'Produto': 'string',
    'Categoria': 'string',
    'Quantidade': 'int64',
    'Cliente_ID': 'int64',
    'Forma_Pagamento': 'string'
})

In [239]:
# Contagem de nulos
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             10000 non-null  datetime64[us]
 1   Loja             10000 non-null  string        
 2   Produto          10000 non-null  string        
 3   Categoria        10000 non-null  string        
 4   Quantidade       10000 non-null  int64         
 5   Preço Unitário   9688 non-null   float64       
 6   Total Venda      7979 non-null   float64       
 7   Cliente_ID       10000 non-null  int64         
 8   Forma_Pagamento  9684 non-null   string        
dtypes: datetime64[us](1), float64(2), int64(2), string(4)
memory usage: 703.3 KB


### Regra de negócio Preço Unitário

In [240]:
# Calcular preço médio de produtos
mediaPreco = round(dados['Preço Unitário'].mean(), 2)
mediaPreco

np.float64(23.24)

In [241]:
# Filtro de valores fora da regra de negócio
dados.loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
27,2023-01-20,Loja A,Cerveja,Bebidas,9,-10.14,NaN,2471,Transferência
31,2023-12-09,Loja C,Pasta de Dente,Higiene,9,-41.96,NaN,8619,Cartão
57,2024-05-26,Loja A,Refrigerante,Bebidas,5,-3.20,NaN,1567,Dinheiro
91,2023-07-13,Loja B,Leite,Alimentos,2,-18.86,NaN,6228,Dinheiro
96,2024-01-17,Loja A,Feijão,Alimentos,3,-12.44,NaN,4016,Pix
...,...,...,...,...,...,...,...,...,...
9830,2024-06-13,Loja A,Detergente,Limpeza,4,-37.82,NaN,2850,Pix
9906,2024-07-19,Loja B,Arroz,Alimentos,6,-33.67,NaN,3402,Pix
9956,2023-03-27,Loja A,Cerveja,Bebidas,1,-15.65,NaN,1950,Cartão
9966,2022-07-18,Loja C,Sabão em Pó,Limpeza,6,-20.57,NaN,9116,Transferência


In [242]:
# Preenchimento de preços vazios
dados['Preço Unitário'] = dados['Preço Unitário'].fillna(mediaPreco)

In [243]:
# Preenchimento de valores incorretos
dados.loc[((dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500.0)), 'Preço Unitário'] = mediaPreco

In [244]:
# Nova verificação de filtros de dados
dados.loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500.0)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento


In [245]:
# Verificação dupla de valores
dados['Preço Unitário'].describe()

count    10000.000000
mean        25.598966
std         13.497159
min          1.500000
25%         14.590000
50%         23.635000
75%         36.920000
max         50.000000
Name: Preço Unitário, dtype: float64

### Regra Total da Venda

In [246]:
# Verificação de dados de venda
dados['Total Venda'].describe()

count    7979.000000
mean      141.407446
std       113.549240
min         1.540000
25%        45.140000
50%       110.800000
75%       212.010000
max       499.300000
Name: Total Venda, dtype: float64

In [247]:
# Verificação de valores incorretos da coluna total de vendas
dados.loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
2,2023-03-21,Loja C,Refrigerante,Bebidas,2,23.24,72.60,5353,Transferência
5,2023-12-27,Loja A,Refrigerante,Bebidas,9,15.64,NaN,8287,Pix
10,2022-03-29,Loja B,Pasta de Dente,Higiene,2,31.21,NaN,1659,Dinheiro
11,2024-10-26,Loja A,Refrigerante,Bebidas,2,16.74,NaN,5087,Cartão
14,2024-06-01,Loja B,Arroz,Alimentos,5,10.27,51.35,4418,Pix
...,...,...,...,...,...,...,...,...,...
9987,2022-08-21,Loja C,Refrigerante,Bebidas,2,43.05,NaN,7156,Transferência
9988,2024-10-20,Loja A,Detergente,Limpeza,2,11.74,NaN,9310,Dinheiro
9989,2024-11-08,Loja C,Cerveja,Bebidas,5,49.33,246.65,5255,Dinheiro
9991,2023-05-04,Loja A,Pasta de Dente,Higiene,6,35.05,NaN,9151,Transferência


In [ ]:
# Preenchimento de valores corretos na coluna seguindo a formúla
dados['Total Venda'] = round(dados['Preço Unitário'] * dados['Quantidade'], 2

In [249]:
# Verificação de valores incorretos da coluna total de vendas
dados.loc[dados['Total Venda'] != (dados['Preço Unitário'] * dados['Quantidade'])]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
14,2024-06-01,Loja B,Arroz,Alimentos,5,10.27,51.35,4418,Pix
15,2023-09-25,Loja B,Cerveja,Bebidas,9,22.04,198.36,7318,Pix
17,2023-08-08,Loja C,Açúcar,Alimentos,9,34.16,307.44,8996,Pix
21,2022-05-28,Loja C,Sabão em Pó,Limpeza,7,26.15,183.05,2164,Transferência
48,2022-01-01,Loja B,Cerveja,Bebidas,5,20.01,100.05,8130,Cartão
...,...,...,...,...,...,...,...,...,...
9953,2022-07-17,Loja A,Refrigerante,Bebidas,3,13.22,39.66,4990,Pix
9984,2023-06-25,Loja A,Sabão em Pó,Limpeza,3,19.92,59.76,5779,Pix
9989,2024-11-08,Loja C,Cerveja,Bebidas,5,49.33,246.65,5255,Dinheiro
9991,2023-05-04,Loja A,Pasta de Dente,Higiene,6,35.05,210.30,9151,Transferência


In [250]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             10000 non-null  datetime64[us]
 1   Loja             10000 non-null  string        
 2   Produto          10000 non-null  string        
 3   Categoria        10000 non-null  string        
 4   Quantidade       10000 non-null  int64         
 5   Preço Unitário   10000 non-null  float64       
 6   Total Venda      10000 non-null  float64       
 7   Cliente_ID       10000 non-null  int64         
 8   Forma_Pagamento  9684 non-null   string        
dtypes: datetime64[us](1), float64(2), int64(2), string(4)
memory usage: 703.3 KB


### Regra de negócio Forma de pagamanto

In [251]:
# Verificando valores vazios na forma de pagamento
dados.loc[dados['Forma_Pagamento'].isnull()]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
35,2022-03-10,Loja B,Refrigerante,Bebidas,4,15.73,62.92,3066,<NA>
60,2024-08-27,Loja A,Feijão,Alimentos,8,4.43,35.44,1723,<NA>
178,2022-03-13,Loja A,Sabão em Pó,Limpeza,9,25.05,225.45,8449,<NA>
226,2024-05-18,Loja B,Açúcar,Alimentos,2,11.95,23.90,6778,<NA>
326,2022-02-14,Loja A,Shampoo,Higiene,2,2.21,4.42,5070,<NA>
...,...,...,...,...,...,...,...,...,...
9931,2023-08-22,Loja B,Shampoo,Higiene,2,1.96,3.92,7775,<NA>
9950,2024-02-13,Loja B,Cerveja,Bebidas,6,3.91,23.46,2499,<NA>
9958,2022-10-11,Loja A,Cerveja,Bebidas,2,6.14,12.28,5182,<NA>
9977,2023-04-28,Loja A,Refrigerante,Bebidas,8,16.49,131.92,0,<NA>


In [252]:
# Preenchimento de valores nulos por valor padrão
dados['Forma_Pagamento'] = dados['Forma_Pagamento'].fillna('Não informado')

In [253]:
# Verificação de valores nulos
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             10000 non-null  datetime64[us]
 1   Loja             10000 non-null  string        
 2   Produto          10000 non-null  string        
 3   Categoria        10000 non-null  string        
 4   Quantidade       10000 non-null  int64         
 5   Preço Unitário   10000 non-null  float64       
 6   Total Venda      10000 non-null  float64       
 7   Cliente_ID       10000 non-null  int64         
 8   Forma_Pagamento  10000 non-null  string        
dtypes: datetime64[us](1), float64(2), int64(2), string(4)
memory usage: 703.3 KB


In [254]:
dados.head(20)

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
0,2022-01-18,Loja B,Refrigerante,Bebidas,3,7.50,22.50,6727,Dinheiro
1,2024-12-29,Loja B,Leite,Alimentos,5,14.74,73.70,2120,Transferência
2,2023-03-21,Loja C,Refrigerante,Bebidas,2,23.24,46.48,5353,Transferência
3,2022-08-03,Loja C,Feijão,Alimentos,4,26.75,107.00,6835,Transferência
4,2022-06-29,Loja C,Feijão,Alimentos,7,30.47,213.29,5824,Dinheiro
5,2023-12-27,Loja A,Refrigerante,Bebidas,9,15.64,140.76,8287,Pix
6,2022-02-20,Loja B,Leite,Alimentos,5,19.59,97.95,4872,Dinheiro
7,2023-05-01,Loja C,Feijão,Alimentos,7,39.36,275.52,5169,Cartão
8,2023-06-13,Loja A,Arroz,Alimentos,9,2.75,24.75,2876,Cartão
9,2023-02-24,Loja C,Sabão em Pó,Limpeza,4,4.29,17.16,3029,Dinheiro


In [255]:
# Contagem de nulos
dados.isnull().sum()

Data               0
Loja               0
Produto            0
Categoria          0
Quantidade         0
Preço Unitário     0
Total Venda        0
Cliente_ID         0
Forma_Pagamento    0
dtype: int64

### Salvando Arquivo Processado

In [256]:
dados.to_csv(arquivo_processado, sep=';', index=False, encoding='utf-8')